In [16]:
import os, tarfile
import re, glob
import pandas as pd
import numpy as np
import zipfile, gzip
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [17]:
df_train = pd.read_csv("/data/private/pdutta/DNABERT_data/prom-300/prom-300/noTATA_249to50.tsv", sep="\t")

In [18]:
df_train

,seq,label,TATA
0,AGCCACGTGAGTCGCTGGGCTATGGGTGGTGGTGGGGGTGAGGGAG...,1,0
1,TACTTCCGGTTTCCACGGAGCTCCGCCCCTTAGGGGGGTTCTCGCT...,1,0
2,GGAGCCTGGTAGGGAGGACAAATCTCTCGAAATCTCAGTTGGCGCT...,1,0
3,GACCTCCTGTGGTGCTTTCAGTTAGAGCCACTCCCTTTACAAACAA...,1,0
4,AATATCTAGGCTTTTAAAATAAGGTGAAAATAAAAGGCAAATTGTC...,1,0
...,...,...,...
53061,TTAGCCTCTAGGCCTCAGTAGTCCCAGGGATCAGTAGCCCTAGAGG...,0,0
53062,ATAGCCGTGCATGTACTCATTCCGCTCGGCAATGAGAAAAAAAAGG...,0,0
53063,CCCCCTCTTAGATTATCCCTGCCGGAGAGAGGAGAGGGGACATAGG...,0,0
53064,CGCACAGAGCATTCCGTCACCCCAGGCCCAGCTCTCCGAACAACAC...,0,0


In [19]:
def seq2kmer(seq, k):
    """
    Convert original sequence to kmers
    
    Arguments:
    seq -- str, original sequence.
    k -- int, kmer of length k specified.
    
    Returns:
    kmers -- str, kmers separated by space
    """
    kmer = [seq[x:x+k] for x in range(len(seq)+1-k)]
    kmers = " ".join(kmer)
    return kmers

In [20]:
def core_prom_35to34(text):
    return text[214:284]

def core_prom_45to44(text):
    return text[204:294]

In [21]:
df_train['CoreProm-35to34']= df_train['seq'].apply(core_prom_35to34)
df_train['CoreProm-45to44']= df_train['seq'].apply(core_prom_45to44)

In [22]:
df_train

,seq,label,TATA,CoreProm-35to34,CoreProm-45to44
0,AGCCACGTGAGTCGCTGGGCTATGGGTGGTGGTGGGGGTGAGGGAG...,1,0,ACTTCCGCAACCAGCTACTCAAAAGCTTCGACTTCCACTTTGGCTT...,GAGAGGCACTACTTCCGCAACCAGCTACTCAAAAGCTTCGACTTCC...
1,TACTTCCGGTTTCCACGGAGCTCCGCCCCTTAGGGGGGTTCTCGCT...,1,0,AGGCGAGAGAGGCGGAGCCGTGGACAGTGGGCGGGAGGCTGCCAAC...,GACGTCATCGAGGCGAGAGAGGCGGAGCCGTGGACAGTGGGCGGGA...
2,GGAGCCTGGTAGGGAGGACAAATCTCTCGAAATCTCAGTTGGCGCT...,1,0,GGGCTCCCTCGGGACTGGGGCGACTGCGCATGCTCGCTGGCCGCGC...,CGGCTGGCCCGGGCTCCCTCGGGACTGGGGCGACTGCGCATGCTCG...
3,GACCTCCTGTGGTGCTTTCAGTTAGAGCCACTCCCTTTACAAACAA...,1,0,GTCAGAGTCCCTGACCATCAGAATCTGCTATGTCACCAAACCTCTC...,AGTGCTGGGAGTCAGAGTCCCTGACCATCAGAATCTGCTATGTCAC...
4,AATATCTAGGCTTTTAAAATAAGGTGAAAATAAAAGGCAAATTGTC...,1,0,GCTCAGTTTGGGCTTGGATGGTAACGTTTATTTTCCTTGGCAGAGA...,GGGAAATGCTGCTCAGTTTGGGCTTGGATGGTAACGTTTATTTTCC...
...,...,...,...,...,...
53061,TTAGCCTCTAGGCCTCAGTAGTCCCAGGGATCAGTAGCCCTAGAGG...,0,0,AAAACCCGACAAATGAGACTTCACTGGAGCTGAGTAGTTGGCTTGC...,GGCTACAATAAAAACCCGACAAATGAGACTTCACTGGAGCTGAGTA...
53062,ATAGCCGTGCATGTACTCATTCCGCTCGGCAATGAGAAAAAAAAGG...,0,0,ATGGCTGATCTCCGGAGTCGCGCCTATCCGGGGTTTTGTGTCCATT...,ACTGGTCAGAATGGCTGATCTCCGGAGTCGCGCCTATCCGGGGTTT...
53063,CCCCCTCTTAGATTATCCCTGCCGGAGAGAGGAGAGGGGACATAGG...,0,0,ACCTCCTGATGTTCCTGACCATGCTGCAGGCTCACGTTAGCGCAGC...,AGTTTCCAGGACCTCCTGATGTTCCTGACCATGCTGCAGGCTCACG...
53064,CGCACAGAGCATTCCGTCACCCCAGGCCCAGCTCTCCGAACAACAC...,0,0,CTTACCTACATCGGCTCCTCTCCGGGAAACCAAACAAGAGTGCTGC...,CCCGCAGCAGCTTACCTACATCGGCTCCTCTCCGGGAAACCAAACA...


In [23]:
df_Prom_nonTATA= pd.DataFrame()
df_coreProm70_nonTATA =pd.DataFrame()
df_coreProm90_nonTATA =pd.DataFrame()

In [24]:
df_Prom_nonTATA['Sequence'] = df_train['seq'].apply(seq2kmer, args=(6,))
df_Prom_nonTATA['Label'] = df_train['label']
df_Prom_nonTATA

,Sequence,Label
0,AGCCAC GCCACG CCACGT CACGTG ACGTGA CGTGAG GTGA...,1
1,TACTTC ACTTCC CTTCCG TTCCGG TCCGGT CCGGTT CGGT...,1
2,GGAGCC GAGCCT AGCCTG GCCTGG CCTGGT CTGGTA TGGT...,1
3,GACCTC ACCTCC CCTCCT CTCCTG TCCTGT CCTGTG CTGT...,1
4,AATATC ATATCT TATCTA ATCTAG TCTAGG CTAGGC TAGG...,1
...,...,...
53061,TTAGCC TAGCCT AGCCTC GCCTCT CCTCTA CTCTAG TCTA...,0
53062,ATAGCC TAGCCG AGCCGT GCCGTG CCGTGC CGTGCA GTGC...,0
53063,CCCCCT CCCCTC CCCTCT CCTCTT CTCTTA TCTTAG CTTA...,0
53064,CGCACA GCACAG CACAGA ACAGAG CAGAGC AGAGCA GAGC...,0


In [25]:
train, test = train_test_split(df_Prom_nonTATA, test_size=0.15)
print(train.groupby(['Label']).size(), test.groupby(['Label']).size())
train.to_csv("/data/private/pdutta/DNABERT_data/nonTATA_Prom-249to50/train.tsv", sep="\t", index=False)
test.to_csv("/data/private/pdutta/DNABERT_data/nonTATA_Prom-249to50/dev.tsv", sep="\t", index=False)

Label
0    22559
1    22547
dtype: int64 Label
0    3974
1    3986
dtype: int64


In [26]:
df_coreProm70_nonTATA['Sequence'] = df_train['CoreProm-35to34'].apply(seq2kmer, args=(6,))
df_coreProm70_nonTATA['Label'] = df_train['label']
df_coreProm70_nonTATA

,Sequence,Label
0,ACTTCC CTTCCG TTCCGC TCCGCA CCGCAA CGCAAC GCAA...,1
1,AGGCGA GGCGAG GCGAGA CGAGAG GAGAGA AGAGAG GAGA...,1
2,GGGCTC GGCTCC GCTCCC CTCCCT TCCCTC CCCTCG CCTC...,1
3,GTCAGA TCAGAG CAGAGT AGAGTC GAGTCC AGTCCC GTCC...,1
4,GCTCAG CTCAGT TCAGTT CAGTTT AGTTTG GTTTGG TTTG...,1
...,...,...
53061,AAAACC AAACCC AACCCG ACCCGA CCCGAC CCGACA CGAC...,0
53062,ATGGCT TGGCTG GGCTGA GCTGAT CTGATC TGATCT GATC...,0
53063,ACCTCC CCTCCT CTCCTG TCCTGA CCTGAT CTGATG TGAT...,0
53064,CTTACC TTACCT TACCTA ACCTAC CCTACA CTACAT TACA...,0


In [27]:
train, test = train_test_split(df_coreProm70_nonTATA, test_size=0.15)
print(train.groupby(['Label']).size(), test.groupby(['Label']).size())
train.to_csv("/data/private/pdutta/DNABERT_data/nonTATA_CoreProm-35to34/train.tsv", sep="\t", index=False)
test.to_csv("/data/private/pdutta/DNABERT_data/nonTATA_CoreProm-35to34/dev.tsv", sep="\t", index=False)

Label
0    22563
1    22543
dtype: int64 Label
0    3970
1    3990
dtype: int64


In [29]:
df_coreProm90_nonTATA['Sequence'] = df_train['CoreProm-45to44'].apply(seq2kmer, args=(6,))
df_coreProm90_nonTATA['Label'] = df_train['label']
df_coreProm90_nonTATA

,Sequence,Label
0,GAGAGG AGAGGC GAGGCA AGGCAC GGCACT GCACTA CACT...,1
1,GACGTC ACGTCA CGTCAT GTCATC TCATCG CATCGA ATCG...,1
2,CGGCTG GGCTGG GCTGGC CTGGCC TGGCCC GGCCCG GCCC...,1
3,AGTGCT GTGCTG TGCTGG GCTGGG CTGGGA TGGGAG GGGA...,1
4,GGGAAA GGAAAT GAAATG AAATGC AATGCT ATGCTG TGCT...,1
...,...,...
53061,GGCTAC GCTACA CTACAA TACAAT ACAATA CAATAA AATA...,0
53062,ACTGGT CTGGTC TGGTCA GGTCAG GTCAGA TCAGAA CAGA...,0
53063,AGTTTC GTTTCC TTTCCA TTCCAG TCCAGG CCAGGA CAGG...,0
53064,CCCGCA CCGCAG CGCAGC GCAGCA CAGCAG AGCAGC GCAG...,0


In [30]:
train, test = train_test_split(df_coreProm90_nonTATA, test_size=0.15)
print(train.groupby(['Label']).size(), test.groupby(['Label']).size())
train.to_csv("/data/private/pdutta/DNABERT_data/nonTATA_CoreProm-45to44/train.tsv", sep="\t", index=False)
test.to_csv("/data/private/pdutta/DNABERT_data/nonTATA_CoreProm-45to44/dev.tsv", sep="\t", index=False)

Label
0    22574
1    22532
dtype: int64 Label
0    3959
1    4001
dtype: int64
